# ORM Has One Reading

### Introduction

In this lesson, we'll begin adding relationship methods to our class.  Remember that in a relational database, we think of our records as being connected.  For example, if we are keeping track of the orders of a customer, we think of how a customer `has_many` orders, and an order `hasone` customer.  So far we have written queries directly in SQL to return the associated records.  In this lesson, we'll add functions to our classes that execute these SQL queries for us.   

### Sending Messages to Instances

Let's again work with Moe's bar.  We can begin by loading the data from our database.  And let's take a look at the tables.

In [1]:
import sqlite3
conn = sqlite3.connect('./moes_bar.db')
cursor = conn.cursor()

import pandas as pd
root_url = "https://raw.githubusercontent.com/jigsawlabs-student/curriculum-images/main/has-many-through-bar/data/"
names = ['bartenders', 'customers', 'drinks', 'orders', 'ingredients', 'ingredients_drinks']
loaded_dfs = [pd.read_csv(f'{root_url}{name}.csv') for name in names]
for index, name in enumerate(names):
    loaded_dfs[index].to_sql(f'{name}', conn, index = False, if_exists = 'replace')
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('bartenders',),
 ('customers',),
 ('drinks',),
 ('orders',),
 ('ingredients',),
 ('ingredients_drinks',)]

Now as we know, with our relational database, we can perform interesting queries, like see all of the orders of a customer and the customer associated with an order.

### Has one queries

We can start with our `has_one` relationship.  For example, we would say that an order `has_one` customer.  After all, there is a foreign_key of `customer_id` on our `orders` table.

In [2]:
class Order:
    __table__ = 'orders'
    columns = ['id', 'customer_id', 'drink_id', 'bartender_id']

class Customer:
    __table__ = 'customers'
    columns = ['id', 'name', 'hometown']


def build_from_record(Class, record):
    if not record: return None
    attr = dict(zip(Class.columns, record))
    obj = Class()
    obj.__dict__ = attr
    return obj

So if we say get an order frmo our database...

In [3]:
cursor.execute('SELECT * FROM orders ORDER BY id DESC LIMIT 1;')
last_order_record = cursor.fetchone()

last_order = build_from_record(Order, last_order_record)

last_order.__dict__

{'id': 9, 'customer_id': 3, 'drink_id': 4, 'bartender_id': 3}

And then want to write a function to return the associated customer, this looks like the following.

In [4]:
def customer_of_order(order):
    cursor.execute('SELECT * FROM customers WHERE id = ?', (order.customer_id,))
    customer_record = cursor.fetchone()
    return build_from_record(Customer, customer_record)

In [6]:
customer = customer_of_order(last_order)
customer.__dict__

{'id': 3, 'name': 'lisa simpson', 'hometown': 'philly'}

Now instead of calling our `customer_of_order` function, it would be nice if we could call `order.customer()`, and our associated customer was automatically returned.  Ok here's how we do it.

In [7]:
class Order:
    __table__ = 'orders'
    columns = ['id', 'customer_id', 'drink_id', 'bartender_id']

    def customer(self):
        cursor.execute('SELECT * FROM customers WHERE id = ?', (self.customer_id,))
        customer_record = cursor.fetchone()
        return build_from_record(Customer, customer_record)

In [8]:
last_order = build_from_record(Order, last_order_record)

customer = last_order.customer()

customer.__dict__

{'id': 3, 'name': 'lisa simpson', 'hometown': 'philly'}

> Ok, so the big change is to now get the `customer_id` from self.  After all, it's our `last_order`, that has the `customer_id`, and then we take this `customer_id` and search through the customer's table to find the customer with that id.

Before moving on, let's spend a little more time on how to come up with these queries.  We start with our order.

In [9]:
last_order.__dict__

{'id': 9, 'customer_id': 3, 'drink_id': 4, 'bartender_id': 3}

And want to find the associated `customer`.  As a starting point, we have access to all of the information on the a particular order, which we see above.  And we want to wind up with a customer.  So this means we connect the two by finding where the order's `customer_id` equals the customer's id.  Then we return an associated customer object.

In [10]:
def customer(self):
    cursor.execute('SELECT * FROM customers WHERE id = ?', (self.customer_id,))
    customer_record = cursor.fetchone()
    return build_from_record(Customer, customer_record)

### Give it a shot

Ok now before moving on, fill in the drink and bartender functions from the order below.

> Try to write it out without referencing the `customer` function above (stretch yourself).

In [22]:
class Order:
    __table__ = 'orders'
    columns = ['id', 'customer_id', 'drink_id', 'bartender_id']

    def drink(self):
        query = 'select * from drinks where drinks.id = ?'
        cursor.execute(query, (self.drink_id,))
        drink = cursor.fetchone()
        return build_from_record(Drink, drink)

    def bartender(self):
        query = 'select * from bartenders where bartenders.id = ?'
        cursor.execute(query, (self.bartender_id,))
        bartender = cursor.fetchone()
        return build_from_record(Bartender, bartender)

class Drink:
    __table__ = 'drinks'
    columns = ['id', 'name', 'calories', 'price', 'alcoholic']

class Bartender:
    __table__ = 'bartenders'
    columns = ['id', 'name', 'hometown', 'birthyear']


def build_from_record(Class, record):
    if not record: return None
    attr = dict(zip(Class.columns, record))
    obj = Class()
    obj.__dict__ = attr
    return obj

In [23]:
last_order = build_from_record(Order, last_order_record)

bartender = last_order.bartender()

bartender.__dict__
# {'id': 3, 'name': 'patty', 'hometown': 'philly', 'birthyear': 1970}

{'id': 3, 'name': 'patty', 'hometown': 'philly', 'birthyear': 1970}

In [24]:
drink = last_order.drink()
drink.__dict__

# {'id': 4, 'name': 'ice cream float', 'calories': 250, 'price': 8, 'alcoholic': 0}

{'id': 4,
 'name': 'ice cream float',
 'calories': 250,
 'price': 8,
 'alcoholic': 0}

### Summary

In this lesson, we practiced writing a `hasone` relationship method.  The key steps are to think through what information we start with and what object we would like to return from the function.  So to write a function like `order.customer()`, we start with the information on that particular order, like `customer_id`, and want to end by returning the `customer` that same `id`.

In [25]:
def customer(self):
    cursor.execute('SELECT * FROM customers WHERE id = ?', (self.customer_id,))
    customer_record = cursor.fetchone()
    return build_from_record(Customer, customer_record)